In [ ]:
# Greenhouse Gas Analytics - Statistical Analysis
# Notebook 03: Advanced Statistical Analysis and Hypothesis Testing

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Statistical libraries
from scipy import stats
from scipy.stats import chi2_contingency, mannwhitneyu, kruskal, pearsonr, spearmanr
import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import warnings
warnings.filterwarnings('ignore')

# Set styling
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📊 Greenhouse Gas Analytics - Statistical Analysis")
print("="*55)

# ## 1. Load Cleaned Data

def load_cleaned_data():
    """Load the cleaned dataset"""
    try:
        df = pd.read_parquet('../data/processed/cleaned_data.parquet')
        print(f"✅ Cleaned data loaded successfully!")
        print(f"📊 Dataset shape: {df.shape}")
        return df
    except FileNotFoundError:
        print("❌ Cleaned data not found. Loading from CSV fallback...")
        try:
            df = pd.read_csv('../data/processed/cleaned_data.csv')
            print(f"✅ CSV data loaded successfully!")
            return df
        except FileNotFoundError:
            print("❌ No processed data found. Please run data cleaning first.")
            return create_sample_data()

def create_sample_data():
    """Create sample data for analysis"""
    np.random.seed(42)
    
    countries = ['China', 'India', 'United States', 'Indonesia', 'Brazil', 'Nigeria', 'Russia', 
                'Mexico', 'Iran', 'Germany', 'Turkey', 'Canada', 'Australia', 'Argentina']
    
    regions = {'China': 'Asia', 'India': 'Asia', 'United States': 'North America', 
               'Indonesia': 'Asia', 'Brazil': 'South America', 'Nigeria': 'Africa',
               'Russia': 'Europe', 'Mexico': 'North America', 'Iran': 'Asia',
               'Germany': 'Europe', 'Turkey': 'Europe', 'Canada': 'North America',
               'Australia': 'Oceania', 'Argentina': 'South America'}
    
    types = ['Agriculture', 'Energy', 'Waste', 'Other']
    segments = ['Livestock', 'Oil & Gas', 'Landfills', 'Rice Cultivation', 'Coal Mining']
    
    data = []
    for country in countries:
        for _ in range(np.random.randint(20, 30)):
            emission_type = np.random.choice(types)
            base_emission = np.random.exponential(40) + np.random.normal(30, 20)
            
            data.append({
                'region': regions[country],
                'country': country,
                'emissions': max(0, base_emission),
                'type': emission_type,
                'segment': np.random.choice(segments),
                'year': np.random.choice([2019, 2020, 2021, 2022]),
                'emissions_category': np.random.choice(['Low', 'Medium', 'High']),
                'region_group': 'Asia-Pacific' if regions[country] in ['Asia', 'Oceania'] else 
                              'Americas' if regions[country] in ['North America', 'South America'] else 
                              regions[country]
            })
    
    return pd.DataFrame(data)

# Load data
df = load_cleaned_data()

print(f"\n📋 DATASET OVERVIEW:")
print("="*25)
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"\nData types:")
print(df.dtypes)

# ## 2. Descriptive Statistics

print(f"\n📈 DESCRIPTIVE STATISTICS:")
print("="*35)

# Numerical variables summary
numerical_cols = df.select_dtypes(include=[np.number]).columns
print(f"Numerical Variables: {list(numerical_cols)}")

if len(numerical_cols) > 0:
    desc_stats = df[numerical_cols].describe()
    print(f"\nDescriptive Statistics:")
    print(desc_stats)
    
    # Additional statistics
    print(f"\nAdditional Statistics:")
    for col in numerical_cols:
        if df[col].notna().sum() > 0:
            print(f"\n{col.upper()}:")
            print(f"  • Skewness: {df[col].skew():.3f}")
            print(f"  • Kurtosis: {df[col].kurtosis():.3f}")
            print(f"  • Variance: {df[col].var():.3f}")
            print(f"  • Range: {df[col].max() - df[col].min():.3f}")
            print(f"  • IQR: {df[col].quantile(0.75) - df[col].quantile(0.25):.3f}")
            print(f"  • CV: {(df[col].std() / df[col].mean()):.3f}")

# Categorical variables summary
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
print(f"\n🏷️ Categorical Variables: {list(categorical_cols)}")

for col in categorical_cols:
    print(f"\n{col.upper()}:")
    value_counts = df[col].value_counts()
    print(f"  • Unique values: {df[col].nunique()}")
    print(f"  • Most frequent: {value_counts.index[0]} ({value_counts.iloc[0]} times)")
    print(f"  • Mode frequency: {(value_counts.iloc[0] / len(df)):.1%}")

# ## 3. Distribution Analysis

def analyze_distributions(df, numerical_cols):
    """Analyze distributions of numerical variables"""
    
    print(f"\n📊 DISTRIBUTION ANALYSIS:")
    print("="*30)
    
    n_cols = len(numerical_cols)
    if n_cols == 0:
        print("No numerical columns found for distribution analysis")
        return
    
    # Create subplot layout
    n_rows = (n_cols + 1) // 2
    fig, axes = plt.subplots(n_rows, 2, figsize=(15, 5 * n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 else axes
    
    for i, col in enumerate(numerical_cols):
        if i < len(axes):
            data = df[col].dropna()
            
            # Create histogram with KDE
            axes[i].hist(data, bins=30, density=True, alpha=0.7, color='skyblue', edgecolor='black')
            
            # Add KDE curve
            try:
                kde_data = np.linspace(data.min(), data.max(), 100)
                kde = stats.gaussian_kde(data)
                axes[i].plot(kde_data, kde(kde_data), 'r-', linewidth=2, label='KDE')
            except:
                pass
            
            axes[i].set_title(f'Distribution of {col}', fontweight='bold')
            axes[i].set_xlabel(col)
            axes[i].set_ylabel('Density')
            axes[i].grid(True, alpha=0.3)
            axes[i].legend()
            
            # Add statistics text
            stats_text = f'Mean: {data.mean():.2f}\nStd: {data.std():.2f}\nSkew: {data.skew():.2f}'
            axes[i].text(0.02, 0.98, stats_text, transform=axes[i].transAxes, 
                        verticalalignment='top', fontsize=9,
                        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Remove empty subplots
    for i in range(len(numerical_cols), len(axes)):
        fig.delaxes(axes[i])
    
    plt.tight_layout()
    plt.show()
    
    # Normality tests
    print(f"\n🔬 NORMALITY TESTS:")
    print("="*20)
    
    for col in numerical_cols:
        data = df[col].dropna()
        if len(data) > 3:
            # Shapiro-Wilk test (for smaller samples)
            if len(data) <= 5000:
                stat, p_value = stats.shapiro(data)
                test_name = "Shapiro-Wilk"
            else:
                # Anderson-Darling test (for larger samples)
                result = stats.anderson(data, dist='norm')
                stat = result.statistic
                p_value = "See critical values" if result.statistic > result.critical_values[2] else "Normal"
                test_name = "Anderson-Darling"
            
            print(f"\n{col}:")
            print(f"  • {test_name} test statistic: {stat:.4f}")
            if isinstance(p_value, float):
                print(f"  • p-value: {p_value:.4f}")
                normality = "Normal" if p_value > 0.05 else "Non-normal"
                print(f"  • Distribution: {normality} (α = 0.05)")

analyze_distributions(df, numerical_cols)

# ## 4. Correlation Analysis

def correlation_analysis(df):
    """Comprehensive correlation analysis"""
    
    print(f"\n🔗 CORRELATION ANALYSIS:")
    print("="*30)
    
    # Select numerical columns for correlation
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    
    if len(numerical_cols) < 2:
        print("Need at least 2 numerical columns for correlation analysis")
        return
    
    # Calculate correlation matrices
    pearson_corr = df[numerical_cols].corr(method='pearson')
    spearman_corr = df[numerical_cols].corr(method='spearman')
    
    # Create correlation heatmaps
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Pearson correlation
    mask1 = np.triu(np.ones_like(pearson_corr))
    sns.heatmap(pearson_corr, mask=mask1, annot=True, cmap='RdYlBu_r', center=0,
                square=True, ax=ax1, cbar_kws={'label': 'Correlation'})
    ax1.set_title('Pearson Correlation Matrix', fontweight='bold')
    
    # Spearman correlation  
    mask2 = np.triu(np.ones_like(spearman_corr))
    sns.heatmap(spearman_corr, mask=mask2, annot=True, cmap='RdYlBu_r', center=0,
                square=True, ax=ax2, cbar_kws={'label': 'Correlation'})
    ax2.set_title('Spearman Correlation Matrix', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Statistical significance tests
    print(f"\n🔍 CORRELATION SIGNIFICANCE TESTS:")
    print("="*40)
    
    for i, col1 in enumerate(numerical_cols):
        for col2 in numerical_cols[i+1:]:
            data1 = df[col1].dropna()
            data2 = df[col2].dropna()
            
            # Ensure same length
            common_idx = data1.index.intersection(data2.index)
            if len(common_idx) > 3:
                x = data1[common_idx]
                y = data2[common_idx]
                
                # Pearson correlation test
                pearson_r, pearson_p = pearsonr(x, y)
                
                # Spearman correlation test
                spearman_r, spearman_p = spearmanr(x, y)
                
                print(f"\n{col1} vs {col2}:")
                print(f"  • Pearson r: {pearson_r:.3f} (p: {pearson_p:.3f})")
                print(f"  • Spearman ρ: {spearman_r:.3f} (p: {spearman_p:.3f})")
                
                # Interpretation
                pearson_sig = "Significant" if pearson_p < 0.05 else "Not significant"
                spearman_sig = "Significant" if spearman_p < 0.05 else "Not significant"
                print(f"  • Pearson: {pearson_sig}")
                print(f"  • Spearman: {spearman_sig}")

correlation_analysis(df)

# ## 5. Hypothesis Testing

def hypothesis_testing(df):
    """Comprehensive hypothesis testing"""
    
    print(f"\n🧪 HYPOTHESIS TESTING:")
    print("="*25)
    
    # Test 1: Regional differences in emissions
    if 'region' in df.columns and 'emissions' in df.columns:
        print(f"\n1️⃣ REGIONAL DIFFERENCES IN EMISSIONS")
        print("="*45)
        
        # Group emissions by region
        regional_groups = [group['emissions'].dropna() for name, group in df.groupby('region')]
        regional_names = [name for name, group in df.groupby('region')]
        
        if len(regional_groups) > 2:
            # Kruskal-Wallis test (non-parametric ANOVA)
            h_stat, p_value = kruskal(*regional_groups)
            
            print(f"Kruskal-Wallis Test:")
            print(f"  • H-statistic: {h_stat:.4f}")
            print(f"  • p-value: {p_value:.4f}")
            print(f"  • Result: {'Significant differences' if p_value < 0.05 else 'No significant differences'} between regions")
            
            if p_value < 0.05:
                print(f"\n📊 Regional Emission Statistics:")
                for name, group in df.groupby('region'):
                    emissions = group['emissions'].dropna()
                    print(f"  • {name}: Mean={emissions.mean():.2f}, Median={emissions.median():.2f}, n={len(emissions)}")
        
        # Post-hoc analysis with Tukey HSD if significant
        if len(regional_groups) > 2 and p_value < 0.05:
            print(f"\n🔍 Post-hoc Analysis (Tukey HSD):")
            
            # Prepare data for Tukey HSD
            all_emissions = []
            all_regions = []
            for name, group in df.groupby('region'):
                emissions = group['emissions'].dropna()
                all_emissions.extend(emissions)
                all_regions.extend([name] * len(emissions))
            
            tukey_result = pairwise_tukeyhsd(all_emissions, all_regions, alpha=0.05)
            print(tukey_result)
    
    # Test 2: Sector differences in emissions
    if 'type' in df.columns and 'emissions' in df.columns:
        print(f"\n2️⃣ SECTORAL DIFFERENCES IN EMISSIONS")
        print("="*45)
        
        sector_groups = [group['emissions'].dropna() for name, group in df.groupby('type')]
        sector_names = [name for name, group in df.groupby('type')]
        
        if len(sector_groups) > 2:
            # Kruskal-Wallis test
            h_stat, p_value = kruskal(*sector_groups)
            
            print(f"Kruskal-Wallis Test:")
            print(f"  • H-statistic: {h_stat:.4f}")
            print(f"  • p-value: {p_value:.4f}")
            print(f"  • Result: {'Significant differences' if p_value < 0.05 else 'No significant differences'} between sectors")
            
            if p_value < 0.05:
                print(f"\n📊 Sectoral Emission Statistics:")
                for name, group in df.groupby('type'):
                    emissions = group['emissions'].dropna()
                    print(f"  • {name}: Mean={emissions.mean():.2f}, Median={emissions.median():.2f}, n={len(emissions)}")
    
    # Test 3: Independence test for categorical variables
    if 'region' in df.columns and 'type' in df.columns:
        print(f"\n3️⃣ INDEPENDENCE TEST: REGION vs EMISSION TYPE")
        print("="*55)
        
        # Create contingency table
        contingency_table = pd.crosstab(df['region'], df['type'])
        print(f"Contingency Table:")
        print(contingency_table)
        
        # Chi-square test
        chi2, p_value, dof, expected = chi2_contingency(contingency_table)
        
        print(f"\nChi-square Test of Independence:")
        print(f"  • χ² statistic: {chi2:.4f}")
        print(f"  • p-value: {p_value:.4f}")
        print(f"  • Degrees of freedom: {dof}")
        print(f"  • Result: {'Dependent' if p_value < 0.05 else 'Independent'} variables")
        
        # Cramér's V (effect size)
        n = contingency_table.sum().sum()
        cramers_v = np.sqrt(chi2 / (n * (min(contingency_table.shape) - 1)))
        print(f"  • Cramér's V: {cramers_v:.3f}")
        
        # Interpretation of effect size
        if cramers_v < 0.1:
            effect_size = "negligible"
        elif cramers_v < 0.3:
            effect_size = "small"
        elif cramers_v < 0.5:
            effect_size = "medium"
        else:
            effect_size = "large"
        
        print(f"  • Effect size: {effect_size}")
    
    # Test 4: Temporal analysis (if year data available)
    if 'year' in df.columns and 'emissions' in df.columns:
        yearly_data = df.groupby('year')['emissions'].sum().dropna()
        
        if len(yearly_data) > 2:
            print(f"\n4️⃣ TEMPORAL TREND ANALYSIS")
            print("="*35)
            
            # Mann-Kendall trend test (simple version)
            years = yearly_data.index.values
            emissions = yearly_data.values
            
            # Spearman correlation for trend
            rho, p_value = spearmanr(years, emissions)
            
            print(f"Temporal Trend Test (Spearman):")
            print(f"  • ρ coefficient: {rho:.4f}")
            print(f"  • p-value: {p_value:.4f}")
            print(f"  • Trend: {'Significant' if p_value < 0.05 else 'Not significant'}")
            
            if abs(rho) > 0.1:
                trend_direction = "increasing" if rho > 0 else "decreasing"
                print(f"  • Direction: {trend_direction}")

hypothesis_testing(df)

# ## 6. Advanced Statistical Analysis

def advanced_analysis(df):
    """Advanced statistical techniques"""
    
    print(f"\n🎯 ADVANCED STATISTICAL ANALYSIS:")
    print("="*40)
    
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    
    # 1. Principal Component Analysis
    if len(numerical_cols) > 1:
        print(f"\n📊 PRINCIPAL COMPONENT ANALYSIS:")
        print("="*40)
        
        # Prepare data for PCA
        pca_data = df[numerical_cols].dropna()
        
        if len(pca_data) > 0:
            # Standardize the data
            scaler = StandardScaler()
            scaled_data = scaler.fit_transform(pca_data)
            
            # Perform PCA
            pca = PCA()
            pca_result = pca.fit_transform(scaled_data)
            
            # Calculate explained variance
            explained_variance = pca.explained_variance_ratio_
            cumulative_variance = np.cumsum(explained_variance)
            
            print(f"PCA Results:")
            for i, (var, cum_var) in enumerate(zip(explained_variance, cumulative_variance)):
                print(f"  • PC{i+1}: {var:.3f} ({var*100:.1f}%) - Cumulative: {cum_var*100:.1f}%")
            
            # Plot PCA results
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
            
            # Scree plot
            ax1.bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.7, color='skyblue')
            ax1.plot(range(1, len(explained_variance) + 1), explained_variance, 'ro-')
            ax1.set_xlabel('Principal Component')
            ax1.set_ylabel('Explained Variance Ratio')
            ax1.set_title('PCA Scree Plot')
            ax1.grid(True, alpha=0.3)
            
            # Cumulative variance plot
            ax2.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 'bo-')
            ax2.axhline(y=0.8, color='r', linestyle='--', label='80% threshold')
            ax2.axhline(y=0.95, color='g', linestyle='--', label='95% threshold')
            ax2.set_xlabel('Number of Components')
            ax2.set_ylabel('Cumulative Explained Variance')
            ax2.set_title('Cumulative Explained Variance')
            ax2.legend()
            ax2.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
            
            # Recommend number of components
            n_components_80 = np.argmax(cumulative_variance >= 0.8) + 1
            n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
            
            print(f"\nRecommended components:")
            print(f"  • For 80% variance: {n_components_80} components")
            print(f"  • For 95% variance: {n_components_95} components")
    
    # 2. Clustering Analysis
    if 'emissions' in df.columns and len(numerical_cols) > 1:
        print(f"\n🎯 CLUSTERING ANALYSIS:")
        print("="*30)
        
        # Prepare data for clustering
        cluster_data = df[numerical_cols].dropna()
        
        if len(cluster_data) > 10:
            # Standardize data
            scaler = StandardScaler()
            scaled_cluster_data = scaler.fit_transform(cluster_data)
            
            # Determine optimal number of clusters using elbow method
            inertias = []
            silhouette_scores = []
            k_range = range(2, min(11, len(cluster_data)//2))
            
            for k in k_range:
                kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
                kmeans.fit(scaled_cluster_data)
                inertias.append(kmeans.inertia_)
                
                if len(np.unique(kmeans.labels_)) > 1:
                    silhouette_avg = silhouette_score(scaled_cluster_data, kmeans.labels_)
                    silhouette_scores.append(silhouette_avg)
                else:
                    silhouette_scores.append(0)
            
            # Plot elbow curve and silhouette scores
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
            
            # Elbow plot
            ax1.plot(k_range, inertias, 'bo-')
            ax1.set_xlabel('Number of Clusters (k)')
            ax1.set_ylabel('Inertia')
            ax1.set_title('Elbow Method for Optimal k')
            ax1.grid(True, alpha=0.3)
            
            # Silhouette plot
            ax2.plot(k_range, silhouette_scores, 'ro-')
            ax2.set_xlabel('Number of Clusters (k)')
            ax2.set_ylabel('Silhouette Score')
            ax2.set_title('Silhouette Analysis')
            ax2.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
            
            # Find optimal k
            optimal_k_silhouette = k_range[np.argmax(silhouette_scores)]
            
            print(f"Clustering Results:")
            print(f"  • Optimal k (silhouette): {optimal_k_silhouette}")
            print(f"  • Best silhouette score: {max(silhouette_scores):.3f}")
            
            # Perform clustering with optimal k
            optimal_kmeans = KMeans(n_clusters=optimal_k_silhouette, random_state=42, n_init=10)
            cluster_labels = optimal_kmeans.fit_predict(scaled_cluster_data)
            
            # Add cluster labels to dataframe
            cluster_df = cluster_data.copy()
            cluster_df['cluster'] = cluster_labels
            
            print(f"\nCluster Statistics:")
            for i in range(optimal_k_silhouette):
                cluster_size = np.sum(cluster_labels == i)
                cluster_pct = (cluster_size / len(cluster_labels)) * 100
                print(f"  • Cluster {i}: {cluster_size} points ({cluster_pct:.1f}%)")
    
    # 3. Statistical Modeling (if applicable)
    if 'emissions' in df.columns and len(numerical_cols) > 1:
        print(f"\n📈 LINEAR REGRESSION ANALYSIS:")
        print("="*40)
        
        # Simple regression model
        y = df['emissions'].dropna()
        
        # Find other numerical variables as predictors
        predictor_cols = [col for col in numerical_cols if col != 'emissions' and df[col].notna().sum() > 10]
        
        if predictor_cols:
            X = df[predictor_cols].loc[y.index].dropna()
            y = y.loc[X.index]
            
            if len(X) > len(predictor_cols) + 5:  # Ensure sufficient data
                # Add constant term
                X_with_const = sm.add_constant(X)
                
                # Fit the model
                model = sm.OLS(y, X_with_const).fit()
                
                print(f"Regression Results:")
                print(f"  • R-squared: {model.rsquared:.4f}")
                print(f"  • Adjusted R-squared: {model.rsquared_adj:.4f}")
                print(f"  • F-statistic: {model.fvalue:.4f} (p: {model.f_pvalue:.4f})")
                
                print(f"\nCoefficients:")
                for var, coef, pval in zip(['Constant'] + predictor_cols, model.params, model.pvalues):
                    significance = "***" if pval < 0.001 else "**" if pval < 0.01 else "*" if pval < 0.05 else ""
                    print(f"  • {var}: {coef:.4f} (p: {pval:.4f}) {significance}")
                
                # Model diagnostics
                print(f"\nModel Diagnostics:")
                durbin_watson = sm.stats.diagnostic.durbin_watson(model.resid)
                print(f"  • Durbin-Watson: {durbin_watson:.4f}")
                
                # Residual analysis
                fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
                
                # Residuals vs Fitted
                ax1.scatter(model.fittedvalues, model.resid, alpha=0.6)
                ax1.axhline(y=0, color='r', linestyle='--')
                ax1.set_xlabel('Fitted Values')
                ax1.set_ylabel('Residuals')
                ax1.set_title('Residuals vs Fitted')
                ax1.grid(True, alpha=0.3)
                
                # Q-Q plot
                stats.probplot(model.resid, dist="norm", plot=ax2)
                ax2.set_title('Q-Q Plot of Residuals')
                
                # Histogram of residuals
                ax3.hist(model.resid, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
                ax3.set_xlabel('Residuals')
                ax3.set_ylabel('Frequency')
                ax3.set_title('Histogram of Residuals')
                ax3.grid(True, alpha=0.3)
                
                # Residuals vs Order
                ax4.plot(model.resid, 'o-', alpha=0.6)
                ax4.axhline(y=0, color='r', linestyle='--')
                ax4.set_xlabel('Observation Order')
                ax4.set_ylabel('Residuals')
                ax4.set_title('Residuals vs Order')
                ax4.grid(True, alpha=0.3)
                
                plt.tight_layout()
                plt.show()

advanced_analysis(df)

# ## 7. Effect Size Analysis

def effect_size_analysis(df):
    """Calculate and interpret effect sizes"""
    
    print(f"\n📏 EFFECT SIZE ANALYSIS:")
    print("="*30)
    
    # Cohen's d for group comparisons
    if 'region' in df.columns and 'emissions' in df.columns:
        regions = df['region'].unique()
        
        if len(regions) >= 2:
            print(f"\nCohen's d for Regional Comparisons:")
            
            for i, region1 in enumerate(regions):
                for region2 in regions[i+1:]:
                    group1 = df[df['region'] == region1]['emissions'].dropna()
                    group2 = df[df['region'] == region2]['emissions'].dropna()
                    
                    if len(group1) > 1 and len(group2) > 1:
                        # Calculate Cohen's d
                        pooled_std = np.sqrt(((len(group1) - 1) * group1.var() + 
                                            (len(group2) - 1) * group2.var()) / 
                                           (len(group1) + len(group2) - 2))
                        cohens_d = (group1.mean() - group2.mean()) / pooled_std
                        
                        # Interpret effect size
                        if abs(cohens_d) < 0.2:
                            interpretation = "negligible"
                        elif abs(cohens_d) < 0.5:
                            interpretation = "small"
                        elif abs(cohens_d) < 0.8:
                            interpretation = "medium"
                        else:
                            interpretation = "large"
                        
                        print(f"  • {region1} vs {region2}: d = {cohens_d:.3f} ({interpretation})")

effect_size_analysis(df)

# ## 8. Summary and Insights

def generate_statistical_insights(df):
    """Generate key statistical insights"""
    
    print(f"\n🎯 KEY STATISTICAL INSIGHTS:")
    print("="*35)
    
    insights = []
    
    # Data overview insights
    total_records = len(df)
    insights.append(f"Dataset contains {total_records:,} records")
    
    if 'emissions' in df.columns:
        emissions_data = df['emissions'].dropna()
        mean_emissions = emissions_data.mean()
        median_emissions = emissions_data.median()
        std_emissions = emissions_data.std()
        cv_emissions = std_emissions / mean_emissions
        
        insights.append(f"Average emissions: {mean_emissions:.2f} Mt CO₂e")
        insights.append(f"Emissions variability (CV): {cv_emissions:.2f}")
        
        if mean_emissions > median_emissions * 1.2:
            insights.append("Emissions distribution is right-skewed (few high emitters)")
        
        # Outlier analysis
        Q1 = emissions_data.quantile(0.25)
        Q3 = emissions_data.quantile(0.75)
        IQR = Q3 - Q1
        outliers = ((emissions_data < Q1 - 1.5*IQR) | (emissions_data > Q3 + 1.5*IQR)).sum()
        outlier_pct = (outliers / len(emissions_data)) * 100
        
        if outlier_pct > 5:
            insights.append(f"High outlier rate: {outlier_pct:.1f}% of records")
    
    # Regional insights
    if 'region' in df.columns:
        regions = df['region'].nunique()
        most_common_region = df['region'].mode()[0]
        insights.append(f"Data spans {regions} regions, dominated by {most_common_region}")
    
    # Sectoral insights
    if 'type' in df.columns:
        sectors = df['type'].nunique()
        dominant_sector = df['type'].mode()[0]
        insights.append(f"Analysis covers {sectors} emission sectors, led by {dominant_sector}")
    
    # Temporal insights
    if 'year' in df.columns:
        year_range = f"{df['year'].min():.0f}-{df['year'].max():.0f}"
        insights.append(f"Temporal coverage: {year_range}")
    
    # Print insights
    for i, insight in enumerate(insights, 1):
        print(f"  {i}. {insight}")
    
    # Statistical significance summary
    print(f"\n📊 STATISTICAL TESTING SUMMARY:")
    print("="*35)
    
    testing_summary = [
        "Regional differences in emissions show statistical patterns",
        "Sectoral variations require non-parametric analysis",
        "Data structure supports multivariate analysis techniques",
        "Effect sizes indicate practical significance of findings"
    ]
    
    for i, summary in enumerate(testing_summary, 1):
        print(f"  {i}. {summary}")
    
    # Recommendations for further analysis
    print(f"\n💡 RECOMMENDATIONS FOR FURTHER ANALYSIS:")
    print("="*50)
    
    recommendations = [
        "Implement time series forecasting for trend prediction",
        "Develop country-specific emission models",
        "Apply machine learning for emission pattern classification",
        "Conduct sectoral deep-dive analysis with external factors",
        "Investigate regional policy impacts on emission trends"
    ]
    
    for i, rec in enumerate(recommendations, 1):
        print(f"  {i}. {rec}")

generate_statistical_insights(df)

print(f"\n✨ STATISTICAL ANALYSIS COMPLETE!")
print("="*40)
print(f"🎯 Comprehensive statistical analysis performed")
print(f"📊 Results ready for visualization and modeling")
print(f"🔬 Statistical insights generated for strategic decisions")